In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz

# Primera iteración

In [149]:
def print_initial_metrics(df):
    print(df.shape)
    print(df.columns)
    print('-----------------------------------------------------------------------------------------')
    print('Nombres diferentes:', len(df['NOMBRE'].unique()))
    print('Códigos diferentes:', len(df['CODIGO'].unique()))
    print('Nits diferentes:', len(df['NIT'].unique()))
    print('Cantidad de datos:', df.shape[0])
    return df.head(2)

In [150]:
entidades = pd.read_csv('../data/silver/entidades_silver.csv', sep=';')
print_initial_metrics(entidades)

(12989, 6)
Index(['NOMBRE', 'CODIGO', 'NIT', 'ORDEN', 'DEPARTAMENTO', 'MUNICIPIO'], dtype='object')
-----------------------------------------------------------------------------------------
Nombres diferentes: 12925
Códigos diferentes: 12989
Nits diferentes: 9246
Cantidad de datos: 12989


,NOMBRE,CODIGO,NIT,ORDEN,DEPARTAMENTO,MUNICIPIO
0,BOYACA EMPRESA DEPARTAMENTAL SERVICIOS PUBLICO...,215001021.0,900297725,TERRITORIAL,BOYACA,TUNJA
1,CUNDINAMARCA ALCALDIA MUNICIPIO VEGA,225402011.0,800128428,TERRITORIAL,CUNDINAMARCA,VEGA


In [238]:
entidades_unicas = {}
df = entidades.sort_values('NOMBRE')
id_counter = 0

grupos_por_entidad = df.groupby(['NOMBRE', 'NIT', 'CODIGO'])
for key_group, group in grupos_por_entidad:
    if len(entidades_unicas.keys()) < 5000:
        flag = False
        for id_entidad in entidades_unicas.keys():
            if not entidades_unicas[id_entidad].empty:
                if (key_group[0] in entidades_unicas[id_entidad]['NOMBRE'].values) or \
                    (key_group[2] in entidades_unicas[id_entidad]['CODIGO'].values):
                    entidades_unicas[id_entidad] = pd.concat([entidades_unicas[id_entidad], 
                                                              df.loc[
                                                                  (df['NOMBRE'] == key_group[0]) &
                                                                  (df['NIT'] == key_group[1]) &
                                                                  (df['CODIGO'] == key_group[2])]])
                    flag = True
                    break
                else:
                    nits_condition = []
                    for nit in entidades_unicas[id_entidad]['NIT'].unique():
                        if pd.notnull(nit) and pd.notnull(key_group[1]) and len(str(nit)) >= 7 and len(str(key_group[1])) >= 7:
                            nits_condition.append(fuzz.partial_ratio(str(key_group[1]), str(nit)))
                    if len(nits_condition) != 0 and sum(nits_condition)/len(nits_condition) >= 89:
                        entidades_unicas[id_entidad] = pd.concat([entidades_unicas[id_entidad], df.loc[
                            (df['NOMBRE'] == key_group[0]) &
                            (df['NIT'] == key_group[1]) &
                            (df['CODIGO'] == key_group[2])]])
                        flag = True
                        break
        if not flag:
            entidades_unicas[id_counter] = df.loc[
                (df['NOMBRE'] == key_group[0]) &
                (df['NIT'] == key_group[1]) &
                (df['CODIGO'] == key_group[2])
            ]
    
            entidades_unicas[id_counter]['ID'] = id_counter
            id_counter += 1

for key in entidades_unicas:
    entidades_unicas[key]['ID'] = key
        
entidades_unicas = pd.concat(entidades_unicas.values())
entidades_unicas = entidades_unicas.sort_values(by='ID')

/tmp/ipykernel_10107/3963280225.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entidades_unicas[id_counter]['ID'] = id_counter
/tmp/ipykernel_10107/3963280225.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entidades_unicas[id_counter]['ID'] = id_counter
/tmp/ipykernel_10107/3963280225.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [240]:
entidades_unicas['ID'].value_counts()[:10]

ID
0       120
2323     94
30       37
3747     37
1177     33
93       30
2584     27
53       24
104      24
142      21
Name: count, dtype: int64

In [244]:
entidades_unicas.to_csv('../data/silver/entidades_unicas.csv',index=False, sep=';')